# Bibliotecas

In [1]:
# Mudar diretório para chamar scripts com funções
import os
os.chdir('/home/francisco/MECAD/2º Ano/Estágio/forcera/scripts/scripts_py')

# Importar funções e flags definidas em flags.py e functions.py
from functions import *
from flags import * 

import psycopg2
from psycopg2 import OperationalError
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from scipy.optimize import curve_fit
import seaborn as sns
from IPython.display import HTML
import webbrowser
from collections import defaultdict
import networkx as nx

plt.rcParams['figure.figsize'] = (7,5)

# Ligação à base de dados

In [2]:
conn = psycopg2.connect(
    host = "contratos-base-gov1.cf87yxnqgph8.eu-central-1.rds.amazonaws.com",
    port = 5432,
    #database = "contratosbasegov",
    user = "contratosbasegov",
    password = "8n9nyeTBFUyCcLJShNrZdPUai2KQkue4")

cur = conn.cursor()

# DataFrame

In [3]:
# Filtrar contratos : ID's concursos públicos com CPV's começados por 72
cpub = (cpv('72', 'Concurso público'))

In [4]:
# Dataframe referente ao conjuntos de contratos públicos celebrados no ramo de consultoria IT 
cp = contratos(cpub)

In [5]:
cp = cp.rename(columns={0:'ID', 1:'NrAnuncio', 2:'PrecoBase', 3:'Prazo', 16:'DataPub', 17:'DataCel', 18:'PrecoContratual',20:'Adjudicante', 22:'Adjudicatária' ,23:'EntConcorrentes', 59:'TotalEfecPrice'})

In [6]:
cp.PrecoBase = cp.PrecoBase.replace('None',0).astype(float)
cp.PrecoContratual = cp.PrecoContratual.replace('None',0).astype(float)
cp.TotalEfecPrice = cp.TotalEfecPrice.replace('None',0).astype(float)

In [7]:
indices = np.where(cp.TotalEfecPrice > 0)[0]
indices

array([  83,  110,  382,  392,  397,  420,  475,  479,  565,  574,  580,
        592,  593,  604,  667,  677,  684,  685,  697,  698,  703,  708,
        734,  736,  745,  788,  797,  807,  869,  909,  922,  923,  969,
        970, 1049, 1078, 1081, 1085, 1086, 1087, 1092, 1095, 1096, 1097,
       1112, 1128, 1129, 1132, 1133, 1162, 1168, 1169, 1171, 1172, 1180,
       1181, 1182, 1184, 1185, 1186, 1187, 1188, 1189, 1198, 1207, 1208,
       1249, 1251, 1289, 1294, 1297, 1298, 1307, 1333, 1354, 1356])

In [16]:
inds = indices[np.where(cp.PrecoContratual[indices] != cp.TotalEfecPrice[indices])[0]]
inds

array([ 479,  909, 1128, 1129, 1251])

In [17]:
print('\t\tPrecoContratual \t PrecoTotalEfetivo')
for i in inds:
    print(cp.ID[i], '\t', cp.PrecoContratual[i], '\t\t', cp.TotalEfecPrice[i])

		PrecoContratual 	 PrecoTotalEfetivo
10005180 	 102200.0 		 30525.0
9565472 	 123830.0 		 97257.93
9497061 	 146136.32 		 19413.68
9497053 	 146136.32 		 61630.56
9317610 	 65664.0 		 20046.5


In [21]:
def rf1(df):
    
    df = df.rename(columns={0:'ID', 1:'NrAnuncio', 2:'PrecoBase', 3:'Prazo', 16:'DataPub', 17:'DataCel', 18:'PrecoContratual',20:'Adjudicante', 22:'Adjudicatária' ,23:'EntConcorrentes', 59:'TotalEfecPrice'})
    df.PrecoBase       = df.PrecoBase.replace('None',0).astype(float)
    df.PrecoContratual = df.PrecoContratual.replace('None',0).astype(float)
    df.TotalEfecPrice  = df.TotalEfecPrice.replace('None',0).astype(float)
    
    indices = np.where(df.TotalEfecPrice > 0)[0]
    inds = indices[np.where(df.PrecoContratual[indices] != df.TotalEfecPrice[indices])[0]]

    return list(df.ID[inds])

In [22]:
rf1(cp)

['10005180', '9565472', '9497061', '9497053', '9317610']